# AML Food challenge:
## A Deep Learning Ensemble-Method for Food Recognition Using Transfer Learning 

## Group 10
Joris van der Vorst, Antonio Javier Samaniego Jurado, Milos Dragojevic

In [ ]:
# Import dependencies
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, model_from_json, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, Activation, Flatten, BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import plot_model
import json
from sklearn.utils import class_weight
import os.path
import fnmatch
import itertools
import functools
from math import floor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

# Addition Inception Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
# Additions for model Antonio
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers


In [ ]:
# Check available model files
!mkdir ../working/models
!ls ../working
!ls ../input/previousfoodmodels/models


# General Settings


In [ ]:
# Settings
img_width, img_height = 224, 224
i3_img_width, i3_img_height = 299, 299
data = pd.read_csv('../input/food-recognition-challenge/train_labels.csv',dtype=str)
display(data.head())
train_data_dir = '../input/food-recognition-challenge/train_set/train_set/'
test_data_dir = '../input/food-recognition-challenge/test_set/test_set/'

# Model weights
checkpoint_vgg= '../working/models/vgg_checkpoint.hdf5'
final_vgg= '../working/models/vgg_final.hdf5'
json_vgg = "../working/models/vgg_model.json"
weights_vgg = "../working/models/vgg_model_weights.h5"

checkpoint_top_i3 = '../working/models/i3_top_checkpoint.hdf5'
checkpoint_full_i3 = '../working/models/i3_full_checkpoint.hdf5'
final_i3= '../working/models/i3_final.hdf5'
json_i3 = "../working/models/i3_model.json"
weights_i3 = "../working/models/i3_model_weights.h5"

# Training history data
history_vgg= '../working/models/vgg_training.pickle'
history_top_i3 = '../working/models/i3_top_training.pickle'
history_full_i3 = '../working/models/i3_full_training.pickle'

submission_file = "submission.csv"

epochs_top = 15 #25
epochs = 25 #100
batch_size = 128
patience_num = 3
min_delta_ea = 0.0005

# Load in model files
vgg_model_file = False # "../input/previousfoodmodels/models/vgg_final.hdf5"
vgg_model_weights = "../input/previousfoodmodels/models/vgg_checkpoint.hdf5"
vgg_model_json_file = "../input/previousfoodmodels/models/vgg_model.json"

i3_model_file = False # "../input/previousfoodmodels/models/i3_final.hdf5"
i3_model_weights = False #"../input/previousfoodmodels/models/i3_top_checkpoint.hdf5"
i3_model_json_file = False #"../input/previousfoodmodels/models/i3_model.json"

#Train-test split
data_h, holdoutSet = train_test_split(data, test_size=0.1, random_state = 21)
#Train-test split
trainingSet, validationSet = train_test_split(data_h, test_size=0.2, random_state = 16)


# Test flow_from_dataframe
sample_csv = pd.read_csv('../input/food-recognition-challenge/sample.csv',dtype=str)


# DataLoaders

In [ ]:
# Create class weights

class_w = class_weight.compute_class_weight('balanced', np.unique(trainingSet["label"]), trainingSet["label"])

#VGG Loaders
def preprocess_input_vgg(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

vgg_train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                       rotation_range=40,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest')

vgg_train_generator = vgg_train_datagen.flow_from_dataframe(
    dataframe= trainingSet,
    x_col="img_name",
    y_col="label",
    directory=train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    seed = 3)

vgg_validation_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
vgg_validation_generator = vgg_validation_test_datagen.flow_from_dataframe(
    dataframe= validationSet,
    x_col="img_name",
    y_col="label",
    directory=train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="categorical",
    seed = 4)

vgg_hold_generator = vgg_validation_test_datagen.flow_from_dataframe(
    dataframe= holdoutSet,
    x_col="img_name",
    y_col="label",
    directory=train_data_dir,
    target_size=(img_width, img_height),
    shuffle=True,
    class_mode="categorical",
    batch_size=batch_size)

vgg_test_generator = vgg_validation_test_datagen.flow_from_dataframe(
    dataframe= sample_csv,
    x_col="img_name",
    y_col="label",
    directory=test_data_dir,
    target_size=(img_width, img_height),
    shuffle=False,
    class_mode=None,
    batch_size=batch_size)

vgg_train_samples = len(vgg_train_generator.filenames)
vgg_validation_samples = len(vgg_validation_generator.filenames)

print("vgg_Train files: ", len(vgg_train_generator.filenames))
print("vgg_Validation files: ", len(vgg_validation_generator.filenames))
print("i3_Holdout files: ", len(vgg_hold_generator.filenames))
print("vgg_Test files: ", len(vgg_test_generator.filenames))

vgg_mapping_indices = vgg_train_generator.class_indices
print(vgg_mapping_indices)


# Inception V3
i3_train_datagen=ImageDataGenerator(rescale=1./255.,
                              rotation_range=40,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest')

i3_train_generator=i3_train_datagen.flow_from_dataframe(
    dataframe= trainingSet,
    directory=train_data_dir,
    x_col="img_name",
    y_col='label',
    batch_size=batch_size,
    seed=5,
    shuffle=True,
    class_mode="categorical",
    target_size=(i3_img_width, i3_img_height))

i3_validation_test_datagen=ImageDataGenerator(rescale=1./255.)

i3_validation_generator=i3_validation_test_datagen.flow_from_dataframe(
    dataframe= validationSet,
    directory=train_data_dir,
    x_col="img_name",
    y_col='label',
    batch_size=batch_size,
    seed=6,
    shuffle=True,
    class_mode="categorical",
    target_size=(i3_img_width, i3_img_height))

i3_hold_generator = i3_validation_test_datagen.flow_from_dataframe(
    dataframe= holdoutSet,
    directory=train_data_dir,
    x_col="img_name",
    y_col="label",
    target_size=(i3_img_width, i3_img_height),
    shuffle=True,
    class_mode="categorical",
    batch_size=batch_size)

i3_test_generator = i3_validation_test_datagen.flow_from_dataframe(
    dataframe= sample_csv,
    x_col="img_name",
    y_col="label",
    directory=test_data_dir,
    target_size=(i3_img_width, i3_img_height),
    shuffle=False,
    class_mode=None,
    batch_size=batch_size)

i3_mapping_indices = i3_train_generator.class_indices

i3_train_samples = len(i3_train_generator.filenames)
i3_validation_samples = len(i3_validation_generator.filenames)

print("i3_Train files: ", len(i3_train_generator.filenames))
print("i3_Validation files: ", len(i3_validation_generator.filenames))
print("i3_Holdout files: ", len(i3_hold_generator.filenames))
print("i3_Test files: ", len(i3_test_generator.filenames))

if i3_mapping_indices == vgg_mapping_indices:
  print("Indices match")
else:
  print("Indices do not match")

# Model VGG16

In [ ]:
#First check is there is a final model
if os.path.isfile(vgg_model_file):
    # Load in final model
    vgg_model = load_model(vgg_model_file)
# Otherwise check if there are checkpoint weights
elif os.path.isfile(vgg_model_weights):
    # If there are weights, see if there is a json file with the structure of the model
    if os.path.isfile(vgg_model_json_file):
        # Load in model structure and weights
        with open(vgg_model_json_file,'r') as f:
            vgg_model = model_from_json(f.read())
        vgg_model.load_weights(vgg_model_weights)
else:

    vgg16 = VGG16(weights='imagenet', include_top = True)
    vgg16.summary()
    x  = vgg16.get_layer('fc2').output
    # x = Flatten(name='flatten2')(x)
    x = Dense(4096, activation='relu', name='d1')(x)
    x = Dropout(0.1)(x)
    x = Dense(4096, activation='relu', name='d2')(x)
    prediction = Dense(80, activation='softmax', name='predictions')(x)

    vgg_model = Model(inputs=vgg16.input, outputs=prediction)
    vgg_model.summary()



# Freeze All Layers Except Bottleneck Layers for Fine-Tuning

for layer in vgg_model.layers:
    if layer.name in ['predictions', 'd1', 'd2', 'block4_conv3', 'block5_conv1', 'block5_conv2', 'block5_conv3']:
        continue
    layer.trainable = False


df = pd.DataFrame(([layer.name, layer.trainable] 
                  for layer in vgg_model.layers), columns=['layer', 'trainable'])

sgd = SGD(lr=1e-4, momentum=0.9, clipvalue = 0.5)

vgg_model.compile(optimizer=sgd, 
                loss='categorical_crossentropy', metrics=['accuracy'])

plot_model(vgg_model, to_file='vgg_model.png',show_shapes=True, 
           show_layer_names=True, dpi = 900)

# serialize model to JSON
model_json = vgg_model.to_json()
with open(json_vgg, "w") as json_file:
    json_file.write(model_json)

# Inception V3 Model

In [ ]:
# Inception V3 model
base_i3_model = InceptionV3(weights='imagenet', include_top=False)

#First check is there is a final model
if os.path.isfile(i3_model_file):
    # Load in final model
    i3_model = load_model(i3_model_file)
# Otherwise check if there are checkpoint weights
elif os.path.isfile(i3_model_weights):
    # If there are weights, see if there is a json file with the structure of the model
    if os.path.isfile(i3_model_json_file):
        # Load in model structure and weights
        with open(i3_model_json_file,'r') as f:
            i3_model = model_from_json(f.read())
        i3_model.load_weights(i3_model_weights)
else:

    # add a global spatial average pooling layer
    x = base_i3_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    # let's add a fully-connected layer (reduced from 1024 to 124)
    x = Dense(124, activation='relu')(x)
    # and a logistic layer, we have 80 classes
    predictions = Dense(80, activation='softmax')(x)

    i3_model = Model(inputs=base_i3_model.input, outputs=predictions)
    
    if i3_model_json_file:
        with open(i3_model_json_file,'r') as f:
            i3_model = model_from_json(f.read())
        i3_model.load_weights(vgg_model_file)

for layer in base_i3_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
i3_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

plot_model(i3_model, to_file='i3_model.png',show_shapes=True, 
           show_layer_names=True, dpi = 900)

# serialize model to JSON
model_json = i3_model.to_json()
with open(json_i3, "w") as json_file:
    json_file.write(model_json)

# Train Models

In [ ]:
# All models should stop if no improvement on validation for a number of epochs
es = EarlyStopping(monitor='val_accuracy', patience= patience_num,
                   verbose=1, min_delta = min_delta_ea)

In [ ]:
# Train Model VGG

#Save the model after every epoch.
mc_vgg = ModelCheckpoint(checkpoint_vgg, monitor='val_accuracy', verbose=1, 
                         save_best_only=True, 
                         save_weights_only=False, mode='auto')

vgg_history = vgg_model.fit_generator(
        vgg_train_generator,
        epochs= epochs,
        validation_data= vgg_validation_generator,
        steps_per_epoch= vgg_train_samples // batch_size,
        validation_steps= vgg_validation_samples // batch_size,
        class_weight=class_w,
        callbacks=[mc_vgg, es])

# serialize weights to HDF5
vgg_model.save_weights(weights_vgg)

vgg_model.save(final_vgg)
print("Saved model to disk")

# Save training data:
with open(history_vgg, 'w') as f:
    pickle.dump(vgg_history.history, f)

In [ ]:
# # Train InceptionV3 top

#Save the model after every epoch.
mc_i3_top = ModelCheckpoint(checkpoint_top_i3, monitor='val_accuracy', verbose=1, 
                            save_best_only=True, 
                            save_weights_only=False, mode='auto')

i3_top_history = i3_model.fit_generator(
        i3_train_generator,
        epochs=  epochs_top,
        validation_data= i3_validation_generator,
        steps_per_epoch=i3_train_samples // batch_size,
        validation_steps=i3_validation_samples // batch_size,
        class_weight=class_w,
        callbacks=[mc_i3_top, es])

# serialize weights to HDF5
i3_model.save_weights(weights_i3)

# Save training data:
with open(history_top_i3, 'wb') as f:
    pickle.dump(i3_top_history.history, f)

In [ ]:
# Train InceptionV3 full

#Save the model after every epoch.
mc_i3_full = ModelCheckpoint(checkpoint_full_i3, monitor='val_accuracy', verbose=1,
                             save_best_only=True, 
                             save_weights_only=False, mode='auto')

# Open up layers
for layer in i3_model.layers[:249]:
    layer.trainable = False
for layer in i3_model.layers[249:]:
    layer.trainable = True

i3_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
                 loss=categorical_crossentropy, metrics=['accuracy'])

i3_full_history = i3_model.fit_generator(
        i3_train_generator,
        epochs= epochs,
        validation_data= i3_validation_generator,
        steps_per_epoch=i3_train_samples // batch_size,
        validation_steps=i3_validation_samples // batch_size,
        class_weight=class_w,
        callbacks=[mc_i3_full, es])


# serialize weights to HDF5
i3_model.save_weights(weights_i3)

i3_model.save(final_i3)
print("Saved model to disk")

# Save training data:
with open(history_full_i3, 'wb') as f:
    pickle.dump(i3_full_history.history, f)

In [ ]:
# Plots VGG16
# Summarize history for accuracy
plt.plot(vgg_history.history['accuracy'])
plt.plot(vgg_history.history['val_accuracy'])
plt.title('VGG16 model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('vgg_acc_1.eps', format='eps', dpi=900)
plt.show()

# Summarize history for loss
plt.plot(vgg_history.history['loss'])
plt.plot(vgg_history.history['val_loss'])
plt.title('VGG16 model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('vgg_val_1.eps', format='eps', dpi=900)
plt.show()

In [ ]:
# Plots Inception V3 top
# Summarize history for accuracy
plt.plot(i3_top_history.history['accuracy'])
plt.plot(i3_top_history.history['val_accuracy'])
plt.title('Inception V3 top model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('i3_top_acc_1.eps', format='eps', dpi=900)
plt.show()

# Summarize history for loss
plt.plot(i3_top_history.history['loss'])
plt.plot(i3_top_history.history['val_loss'])
plt.title('Inception V3 top model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('i3_top_val_1.eps', format='eps', dpi=900)
plt.show()

# Plots I3 full
# Summarize history for accuracy
plt.plot(i3_full_history.history['accuracy'])
plt.plot(i3_full_history.history['val_accuracy'])
plt.title('Inception V3 full model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('i3_acc_1.eps', format='eps', dpi=900)
plt.show()

# Summarize history for loss
plt.plot(i3_full_history.history['loss'])
plt.plot(i3_full_history.history['val_loss'])
plt.title('Inception V3 full model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.tight_layout()
plt.savefig('i3_val_1.eps', format='eps', dpi=900)
plt.show()

# Evaluate models

In [ ]:
# Use best validation checkpoints for predictions:
if os.path.isfile(checkpoint_vgg):
    vgg_model.load_weights(checkpoint_vgg)
if os.path.isfile(checkpoint_full_i3):
    i3_model.load_weights(checkpoint_full_i3)

In [ ]:
vgg_evaluate = vgg_model.evaluate_generator(vgg_hold_generator,verbose=1)
print(vgg_evaluate)
print(vgg_model.metrics_names)

i3_evaluate = i3_model.evaluate_generator(i3_hold_generator,verbose=1)
print(i3_evaluate)

total_performance = vgg_evaluate[1] + i3_evaluate[1]
print(total_performance)
weigth_vgg = vgg_evaluate[1]/total_performance
weigth_i3 = i3_evaluate[1]/total_performance

print(f"""Weight VGG Model: {weigth_vgg}\nWeight Inception V3 Model: {weigth_i3}\n""")

In [ ]:
print(weigth_vgg + weigth_i3)

# Predictions

In [ ]:
# VGG
# Reset to ensure files are loaded in the correct order
vgg_test_generator.reset()
# Genarate predictions
vgg_predict = vgg_model.predict_generator(vgg_test_generator, verbose = 1)

# Test if amount of predictions match the number of examples
print("Amount of predictions match the number of examples: ",
      len(vgg_predict) == sample_csv.shape[0])

# Inception V3
# Reset to ensure files are loaded in the correct order
i3_test_generator.reset()
# Genarate predictions
i3_predict = i3_model.predict_generator(i3_test_generator, verbose = 1)


# Test if amount of predictions match the number of examples
print("Amount of predictions match the number of examples: ",
      len(vgg_predict) ==len(i3_predict)  == sample_csv.shape[0])

print("All indices match: ",
      vgg_mapping_indices == i3_mapping_indices)

print("All filenames match: ",
      vgg_test_generator.filenames == i3_test_generator.filenames)

# Match highest scoring column index to label
labels = dict((v,k) for k,v in vgg_mapping_indices.items())
# Add labels of both models together
total_predict = (vgg_predict*weigth_vgg  + i3_predict*weigth_i3)

print(total_predict.shape)
predictions = [labels[k] for k in np.argmax(total_predict,axis=1)]

prediction = pd.DataFrame({"img_name": vgg_test_generator.filenames, "label":predictions})
display(prediction.head())
# Save submission as csv
prediction.to_csv(submission_file, index = False)